In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import yfinance as yf
from datetime import datetime
import json

In [2]:
def get_price_data(tickers, start_date=None, end_date=None, period="1y", interval="1d"):
    """
    Fetch historical price data for pairs from Yahoo Finance.
    
    Parameters:
    -----------
    tickers : list
        List of ticker symbols to fetch data for
    start_date : str, optional
        Start date in 'YYYY-MM-DD' format. If None, uses period instead.
    end_date : str, optional
        End date in 'YYYY-MM-DD' format. If None, uses current date.
    period : str, optional
        Time period to download data for if start_date is None. 
        Valid values: '1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max'
    interval : str, optional
        Data interval. Valid values: '1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo'
    
    Returns:
    --------
    pd.DataFrame
        DataFrame containing closing prices for all tickers
    """
    if end_date is None:
        end_date = datetime.now().strftime('%Y-%m-%d')
    
    if start_date is None:
        # Use period parameter
        data = yf.download(tickers, period=period, interval=interval, group_by='ticker', auto_adjust=True)
    else:
        data = yf.download(tickers, start=start_date, end=end_date, interval=interval, group_by='ticker', auto_adjust=True)
    
    # Extract closing prices
    if len(tickers) > 1:
        prices = pd.DataFrame()
        for ticker in tickers:
            prices[ticker] = data[ticker]['Close']
    else:
        prices = pd.DataFrame(data['Close'])
        prices.columns = tickers
    
    return prices

In [3]:
def compute_features(price_A: pd.Series, price_B: pd.Series, window: int = 20) -> pd.DataFrame:
    # Static beta estimation using statsmodels OLS instead of LinearRegression
    X = sm.add_constant(price_B)
    model = sm.OLS(price_A, X).fit()

    beta = model.params.iloc[1]  # Slope coefficient
    alpha = model.params.iloc[0]  # Intercept
    
    # Calculate the spread with alpha correction
    spread = price_A - beta * price_B - alpha

    # Z-score of spread
    spread_mean = spread.rolling(window).mean()
    spread_std = spread.rolling(window).std()
    spread_z = (spread - spread_mean) / spread_std

    # Volatility of spread
    spread_vol = spread.rolling(window).std()

    # Half-life of mean reversion
    def calc_halflife(series):
        lagged = series.shift(1)
        delta = series - lagged
        lagged = sm.add_constant(lagged.dropna())
        model = sm.OLS(delta.dropna(), lagged).fit()
        halflife = -np.log(2) / model.params.iloc[1]  # Using index 1 for the coefficient
        return halflife

    halflife_series = spread.rolling(window).apply(calc_halflife, raw=False)

    # Bollinger %B
    upper_band = spread_mean + 2 * spread_std
    lower_band = spread_mean - 2 * spread_std
    bollinger_pct = (spread - lower_band) / (upper_band - lower_band)

    # Rolling beta
    def rolling_beta(a, b, window):
        betas = []
        for i in range(len(a)):
            if i < window:
                betas.append(np.nan)
                continue
            y = a[i-window:i]
            x = b[i-window:i]
            cov = np.cov(y, x)[0, 1]
            var = np.var(x)
            betas.append(cov / var if var != 0 else np.nan)
        return pd.Series(betas, index=a.index)

    rolling_beta_est = rolling_beta(price_A, price_B, window)

    # Volatility ratio
    vol_A = price_A.diff().rolling(window).std()
    vol_B = price_B.diff().rolling(window).std()
    vol_ratio = vol_A / vol_B

    # Lead-lag difference using percentage change
    lead_lag_diff = price_A.pct_change().shift(1) - price_B.pct_change()

    # Combine into DataFrame
    features = pd.DataFrame({
        'price_A': price_A,
        'price_B': price_B,
        'spread': spread,
        'spread_z': spread_z,
        'spread_vol': spread_vol,
        'halflife': halflife_series,
        'bollinger_pct': bollinger_pct,
        'rolling_beta': rolling_beta_est,
        'vol_ratio': vol_ratio,
        'lead_lag_diff': lead_lag_diff,
        'hedge_ratio': beta,
        'alpha': alpha
    })

    features['date'] = features.index

    # Drop rows with NaNs
    features = features.dropna()

    # Add target variable for next day's spread only
    features['next_day_spread'] = features['spread'].shift(-1)

    # Drop the last row as it will have NaNs in the target column
    features = features.dropna()

    return features

In [4]:
def compute_features_for_pair(ticker_A, ticker_B, start_date=None, end_date=None, period="1y", window=20, formation_year=None):
    """
    Download price data for a stock pair and compute features.
    
    Parameters:
    -----------
    ticker_A, ticker_B : str
        Stock ticker symbols
    start_date, end_date : str, optional
        Start and end dates in 'YYYY-MM-DD' format
    period : str, optional
        Time period if start_date is None
    window : int, optional
        Rolling window size for feature calculation
    formation_year : int, optional
        The formation year for the data, to be added as a column in the DataFrame
        
    Returns:
    --------
    pd.DataFrame
        DataFrame containing all computed features
    """
    # Fetch data
    tickers = [ticker_A, ticker_B]
    prices = get_price_data(tickers, start_date, end_date, period)
        
    # Compute features
    features = compute_features(prices[ticker_A], prices[ticker_B], window)
    
    # Add the formation_year column
    if formation_year is not None:
        features['formation_year'] = formation_year
    
    return features

In [7]:
# Load yearly_cointegrations from the JSON file
with open("yearly_cointegrations.json", "r") as json_file:
    yearly_cointegrations = json.load(json_file)

# Collect data for all pairs
all_features = pd.DataFrame()

for start_year, co_pairs_list in yearly_cointegrations.items():
    start_date = f"{start_year}-01-01"
    end_date = f"{int(start_year) + 1}-01-01"  # Convert to int for arithmetic
    print(f"\n=== Running feature engineering for {start_date} to {end_date} ===")

    for pair_data in co_pairs_list:
        tickerA, tickerB = pair_data[0]  # Extract the pair (tickerA, tickerB)
        print(f"  -> Processing pair: {tickerA}-{tickerB}")
        try:
            # Compute features for the pair
            pair_features = compute_features_for_pair(
                ticker_A=tickerA, 
                ticker_B=tickerB, 
                start_date=start_date, 
                end_date=end_date,
                formation_year=int(start_year)  # Pass the start_year as formation_year
            )
            pair_features['pair'] = f"{tickerA}-{tickerB}"  # Add pair identifier
            all_features = pd.concat([all_features, pair_features], ignore_index=True)
        except Exception as e:
            print(f"    Error processing {tickerA}-{tickerB}: {str(e)}")

print("\nFeature engineering complete.")
print("Total rows in all_features:", len(all_features))

# Optionally, save to CSV
all_features.to_csv("yearly_pairs_features.csv", index=False)

[*********************100%***********************]  2 of 2 completed


=== Running feature engineering for 2019-01-01 to 2020-01-01 ===
  -> Processing pair: ACN-MKC



[*********************100%***********************]  2 of 2 completed

  -> Processing pair: AMT-VMC



[*********************100%***********************]  2 of 2 completed

...


[*********************100%***********************]  2 of 2 completed

  -> Processing pair: IR-TDG



[*********************100%***********************]  2 of 2 completed

  -> Processing pair: KIM-NSC



Feature engineering complete.
Total rows in all_features: 29332


In [8]:
# Load yearly_cointegrations from the JSON file
with open("yearly_cointegrations.json", "r") as json_file:
    yearly_cointegrations = json.load(json_file)

# Collect data for all pairs
all_features = pd.DataFrame()

for year, co_pairs_list in yearly_cointegrations.items():
    if year != "2023":
        continue  # Skip all years except 2023
    
    start_year = 2024
    
    # Extend the start date to include the rolling window buffer
    buffer_start_date = f"{int(start_year) - 1}-12-01"  # 30 days before 2024-01-01
    target_start_date = f"{start_year}-01-01"
    end_date = f"{int(start_year) + 1}-01-01"

    print(f"\n=== Running feature engineering for {target_start_date} to {end_date} ===")

    for pair_data in co_pairs_list:
        tickerA, tickerB = pair_data[0]  # Extract the pair (tickerA, tickerB)
        print(f"  -> Processing pair: {tickerA}-{tickerB}")
        try:
            # Compute features for the pair
            pair_features = compute_features_for_pair(
                ticker_A=tickerA, 
                ticker_B=tickerB, 
                start_date=buffer_start_date,  # Use the buffer start date
                end_date=end_date,
                formation_year=int(start_year)  # Pass the start_year as formation_year
            )
            
            # Filter features to only include the target date range
            pair_features = pair_features.loc[target_start_date:end_date]
            
            pair_features['pair'] = f"{tickerA}-{tickerB}"  # Add pair identifier
            all_features = pd.concat([all_features, pair_features], ignore_index=True)
        except Exception as e:
            print(f"    Error processing {tickerA}-{tickerB}: {str(e)}")

print("\nFeature engineering complete.")
print("Total rows in all_features:", len(all_features))

# Optionally, save to CSV
all_features.to_csv("yearly_pairs_features_test.csv", index=False)

[*********************100%***********************]  2 of 2 completed


=== Running feature engineering for 2024-01-01 to 2025-01-01 ===
  -> Processing pair: EXPD-MLM



[*********************100%***********************]  2 of 2 completed

  -> Processing pair: CMS-RTX



[*********************100%***********************]  2 of 2 completed

...

  -> Processing pair: BAX-TECH



[*********************100%***********************]  2 of 2 completed

  -> Processing pair: IR-TDG



[*********************100%***********************]  2 of 2 completed

  -> Processing pair: KIM-NSC

Feature engineering complete.
Total rows in all_features: 4267
